In [22]:
import random
import numpy as np


# Define Sudoku Letters puzzle
# Define the Sudoku puzzle
puzzle = np.array([
    ["w", "", "r", "o"],
    ["o", "r", "", "d"],
    ["r", "o", "", ""],
    ["", "", "o", "r"]
])

# Step 1: Initialize population
# Define the puzzle grid size
GRID_SIZE = 4

# Define the target numbers
TARGET_LETTERS = ["o", "r", "w", "d"]


def generate_individual(puzzle):
    # Generate a random individual (grid) while preserving the initial values from the puzzle
    individual = np.copy(puzzle)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if individual[i][j] == 0:
                individual[i][j] = random.choice(TARGET_LETTERS)
    return individual

# Step 2: Define fitness function
def fitness(individual, puzzle):
    # Calculate the fitness score of an individual
    score = 0

    # Check each row
    for row in individual:
        if len(set(row)) == GRID_SIZE:
            score += 1

    # Check each column
    for j in range(GRID_SIZE):
        column = [individual[i][j] for i in range(GRID_SIZE)]
        if len(set(column)) == GRID_SIZE:
            score += 1

    # Check each 3x3 sub-grid
    for i in range(0, GRID_SIZE, 3):
        for j in range(0, GRID_SIZE, 3):
            subgrid = individual[i:i+3, j:j+3].flatten()
            if len(set(subgrid)) == GRID_SIZE:
                score += 1

    # Preserve the initial values from the puzzle
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if puzzle[i][j] != 0:
                if individual[i][j] == puzzle[i][j]:
                    score += 1

    return score

# Step 3 Selection method: Tournament selection
def tournament_selection(population,tournament_size):
    tournament_size = 2
    parents = []
    for _ in range(len(population)):
        # Randomly select tournament_size individuals from the population
        tournament = random.sample(population, tournament_size)
        # Choose the individual with the highest fitness score as the winner
        winner = max(tournament, key=lambda individual: fitness(individual, puzzle))
        parents.append(winner)
    return parents


# Step 3: Define crossover function
def crossover(parent1, parent2):
    # Perform crossover between two parents to produce two offspring
    point1 = random.randint(1, GRID_SIZE - 2)   
    point2 = random.randint(1, GRID_SIZE - 2)
    offspring1 = np.copy(parent1)
    offspring2 = np.copy(parent2)
    offspring1[:point1], offspring1[point1:] = parent2[:point1], parent1[point1:]
    offspring2[:point2], offspring2[point2:] = parent1[:point2], parent2[point2:]
    return offspring1, offspring2

# Step 4: Define mutate function
def mutate(individual, puzzle):
    # Perform mutation on an individual while preserving the initial values from the puzzle
    mutated_individual = np.copy(individual)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if puzzle[i][j] == "" and random.random() < 0.1:  # Mutation rate: 10% for empty cells only
                mutated_individual[i][j] = random.choice(TARGET_LETTERS)
    return mutated_individual

# Step 5: Define replacement method: Elitism 
def replacement(population, offspring):
    population.extend(offspring)
    population.sort(key=lambda individual: fitness(individual, puzzle), reverse=True)
    return population[:len(population)//2]

# Step 6: Define genetic algorithm
def genetic_algorithm(puzzle, population_size, generations):
    # Initialize the population
    population = [generate_individual(puzzle) for _ in range(population_size)]

    # Store the grids of each iteration
    iteration_grids = []

    for iteration in range(generations):
        # Evaluate the fitness of each individual
        fitness_scores = [fitness(individual, puzzle) for individual in population]
        
        # Select parents for reproduction by tournament selection
        parents = random.choices(population, weights=fitness_scores, k=population_size)

        # Create the next generation by crossover and mutate
        next_generation = []
        for i in range(0, population_size, 2):
            parent1 = parents[i]
            parent2 = parents[i + 1]
            offspring1, offspring2 = crossover(parent1, parent2)
            next_generation.extend([mutate(offspring1, puzzle), mutate(offspring2, puzzle)])

        # Replace the current population with the next generation
        population = next_generation
        
        # Print the iteration and the best fitness score
        best_fitness = max(fitness_scores)
        print(f"Iteration {iteration + 1}: Best Fitness = {best_fitness}")

        # Append the current grid to the iteration_grids list
        iteration_grids.append(population[0])

    # Return the best individual (solution)
    best_individual = max(population, key=lambda x: fitness(x, puzzle))
    return best_individual, iteration_grids

def is_valid(best_individual):
    # Check if the matrix is valid
    for i in range(GRID_SIZE):
        if len(set(best_individual[i])) != GRID_SIZE:
            return False
        if len(set(best_individual[:, i])) != GRID_SIZE:
            return False
    return True

def generate_matrix():
    # Generate a random matrix 
    matrix = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
    LETTERS = TARGET_LETTERS
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            random_letter = random.choice(LETTERS)
            matrix[i][j] = random_letter
            LETTERS.remove(random_letter)
    return matrix

def generate_valid_matrix():
    # Generate a valid matrix
    while True:
        matrix = generate_matrix()
        if is_valid(matrix):
            return matrix


def check_duplicate_letters(matrix):
    # Check if there are duplicate letters in any row or column
    for i in range(GRID_SIZE):
        row = matrix[i, :]
        column = matrix[:, i]
        if len(set(row)) != GRID_SIZE or len(set(column)) != GRID_SIZE:
            return True  # Solution is invalid
    return False  # Solution is valid

        
# Step 6: Termination Solve Sudoku puzzle
# Solve the Sudoku puzzle
generations_num = 100
population_num = 1000
solution, iteration_grids = genetic_algorithm(puzzle, population_size=population_num, generations=generations_num)

# Step 7: Output the best solution
# Check if the solution is valid and print the appropriate message
if check_duplicate_letters(solution):
    text = f"Based on the genetic operators with {generations_num} generations is with a population of: {population_num} \n no valid solution was found where no letter was repeated in the same row or column. \n The best solution found was the following:"
    print(text)
    print(solution)
else:
    text = f"The Best Solution for the Sudoku Letters problem after {generations_num} generations is with a population of: {population_num}"
    print(text)
    print(solution)

# Print the grids of each iteration
print("Iteration Grids:")
for i, grid in enumerate(iteration_grids):
    print(f"Iteration {i+1}:")
    print(grid)
    print()

Iteration 1: Best Fitness = 21
Iteration 2: Best Fitness = 22
Iteration 3: Best Fitness = 23
Iteration 4: Best Fitness = 23
Iteration 5: Best Fitness = 23
Iteration 6: Best Fitness = 23
Iteration 7: Best Fitness = 23
Iteration 8: Best Fitness = 23
Iteration 9: Best Fitness = 23
Iteration 10: Best Fitness = 23
Iteration 11: Best Fitness = 23
Iteration 12: Best Fitness = 23
Iteration 13: Best Fitness = 23
Iteration 14: Best Fitness = 23
Iteration 15: Best Fitness = 23
Iteration 16: Best Fitness = 23
Iteration 17: Best Fitness = 23
Iteration 18: Best Fitness = 23
Iteration 19: Best Fitness = 23
Iteration 20: Best Fitness = 23
Iteration 21: Best Fitness = 23
Iteration 22: Best Fitness = 23
Iteration 23: Best Fitness = 23
Iteration 24: Best Fitness = 23
Iteration 25: Best Fitness = 23
Iteration 26: Best Fitness = 23
Iteration 27: Best Fitness = 23
Iteration 28: Best Fitness = 23
Iteration 29: Best Fitness = 23
Iteration 30: Best Fitness = 23
Iteration 31: Best Fitness = 22
Iteration 32: Bes